In [1]:
import sys
import gensim
import codecs
import json
from gensim.models.keyedvectors import Word2VecKeyedVectors
from gensim.models import KeyedVectors
import numpy as np
import random
import sklearn
from sklearn import model_selection
from sklearn import cluster
from sklearn import metrics
from sklearn.manifold import TSNE
from sklearn.svm import LinearSVC, SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction import DictVectorizer
import numpy as np
from docopt import docopt
import torch
from transformers import *
import pickle
from tqdm import tqdm

import scipy
from scipy import linalg
from scipy import sparse
from scipy.stats.stats import pearsonr
import tqdm
import matplotlib
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDClassifier, SGDRegressor, Perceptron, LogisticRegression


import warnings
warnings.filterwarnings("ignore")

import pickle
from collections import defaultdict, Counter
from typing import List, Dict

import torch
from torch import utils


import pandas as pd

import time
from mlxtend.math import vectorspace_orthonormalization

2023-05-22 14:08:31.818308: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-22 14:08:32.352283: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/leiding/pyotrch/lib/python3.8/site-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/home/leiding/pyotrch/lib/python3.8/site-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from trans

In [2]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [3]:
def load_dataset(path):
    
    with open(path, "rb") as f:
        
        data = pickle.load(f)
    return data

In [4]:
path = "../data/deepmoji"
tags = ['pos_pos','pos_neg','neg_pos','neg_neg']
clss = [[1,1],[1,0],[0,1],[0,0]]
sets = ['test','dev','train']

In [5]:
def preprocess(path, subset, portions):
    X = np.load(path +'/'+subset+'/'+tags[0]+".npy")
    X = X[:portions[0]]
    Y_sem = clss[0][0]*np.ones(X.shape[0])
    Y_race = clss[0][1]*np.ones(X.shape[0])
    
    
    for i in range(1,len(tags)):
        X_new = np.load(path +'/'+subset+'/'+tags[i]+".npy")
        X_new = X_new[:portions[i]]
        X = np.r_[X, X_new]
        Y_sem_new = clss[i][0]*np.ones(X_new.shape[0])
        Y_sem = np.r_[Y_sem, Y_sem_new]
        Y_race_new = clss[i][1]*np.ones(X_new.shape[0])
        Y_race = np.r_[Y_race, Y_race_new]

    print(X.shape)

    np.random.seed(0)
    np.random.shuffle(X)

    np.random.seed(0)
    np.random.shuffle(Y_sem)

    np.random.seed(0)
    np.random.shuffle(Y_race)
    
    return X, Y_sem, Y_race

In [6]:
X_train, Y_sem_train, Y_race_train = preprocess(path, 'train', [40000,10000,10000,40000])

(100000, 2304)


In [7]:
X_test, Y_sem_test, Y_race_test = preprocess(path, 'test', [2000, 2000, 2000, 2000])

(7998, 2304)


In [8]:
def TPR(Y_p, Y_t, tag):
    return np.mean(Y_p[Y_t == tag] == tag)

In [9]:
def get_TPR_gap(Y_sem_pred, Y_sem_true, Y_race):
    gap1 = TPR(Y_sem_pred[Y_race == 0], Y_sem_true[Y_race == 0], 0) - TPR(Y_sem_pred[Y_race == 1], Y_sem_true[Y_race == 1], 0)
    gap2 = TPR(Y_sem_pred[Y_race == 0], Y_sem_true[Y_race == 0], 1) - TPR(Y_sem_pred[Y_race == 1], Y_sem_true[Y_race == 1], 1)
    rms = np.sqrt(0.5*gap1**2 + 0.5*gap2**2)
    return rms

In [10]:
clf = LogisticRegression(warm_start = True, penalty = 'l2',
                         solver = "saga", multi_class = 'multinomial', fit_intercept = False,
                         verbose = 5, n_jobs = 90, random_state = 1, max_iter = 7)
clf.fit(X_train, Y_sem_train)

[Parallel(n_jobs=90)]: Using backend ThreadingBackend with 90 concurrent workers.


max_iter reached after 16 seconds


[Parallel(n_jobs=90)]: Done   1 out of   1 | elapsed:   16.4s finished


LogisticRegression(fit_intercept=False, max_iter=7, multi_class='multinomial',
                   n_jobs=90, random_state=1, solver='saga', verbose=5,
                   warm_start=True)

In [11]:
clf.score(X_test, Y_sem_test)

0.7119279819954989

In [12]:
Y_sem_pred_orig = clf.predict(X_test)

In [13]:
get_TPR_gap(Y_sem_pred_orig, Y_sem_test, Y_race_test)

0.38949996396878994

In [14]:
def get_TPR(y_main, y_hat_main, y_protected):
    
    all_y = list(Counter(y_main).keys())
    
    protected_vals = defaultdict(dict)
    for label in all_y:
        for i in range(2):
            used_vals = (y_main == label) & (y_protected == i)
            y_label = y_main[used_vals]
            y_hat_label = y_hat_main[used_vals]
            protected_vals['y:{}'.format(label)]['p:{}'.format(i)] = (y_label == y_hat_label).mean()
            
    diffs = {}
    for k, v in protected_vals.items():
        vals = list(v.values())
        diffs[k] = vals[0] - vals[1]
    return protected_vals, diffs

In [15]:
def rms(arr):
    return np.sqrt(np.mean(np.square(arr)))

In [16]:
_, diffs = get_TPR(Y_sem_test, Y_sem_pred_orig,  Y_race_test)

In [17]:
rms(list(diffs.values()))

0.38949996396878994

In [19]:
def slice_freq(Y, interval):
    y_f = pd.Series(Y)
    a = pd.cut(y_f,interval)
    b=a.value_counts()
    frequency = b.sort_index().values
    return frequency

In [20]:
def SDR_multi_new(X_train, Y, K, h):

    n = X_train.shape[0]
    d_x = X_train.shape[1]
    d_y = Y.shape[1]
    
    X = X_train

    interval = np.linspace(0, 1, h+1,endpoint=True)
    Gamma_PMS = np.zeros([d_x,d_x])
    for i in range(d_y):  
        #interval = np.linspace(np.min(Y[:,i])-0.1, np.max(Y[:,i])+0.1, h+1,endpoint=True)
        freq = slice_freq(Y[:,i], interval)
        Gamma = np.zeros([d_x,d_x])
        for j in range(h):
            ph = freq[j]/n
            if ph == 0:
                mh = np.zeros(d_x)
            else:
                index = np.where((Y[:,i]>=interval[j])&(Y[:,i]<=interval[j+1]))
                mh = (1/(n*ph))*np.sum(X[index,:],0)
            Gamma = Gamma + ph*np.dot(mh.T,mh)
        la_g, v_g = np.linalg.eig(Gamma)
        la_g = la_g.real
        Gamma_PMS = Gamma_PMS + la_g[0]*Gamma
    la_PMS, v_PMS = np.linalg.eig(Gamma_PMS)
    v_PMS = v_PMS.real
    b1 = v_PMS[:,:K] + 0.001*np.random.rand(d_x,K)
    b2 = v_PMS[:,K:] + 0.001*np.random.rand(d_x,d_x-K)
    #print(b1)
    beta1 = get_rowspace_projection(b1.T) #vectorspace_orthonormalization(v_PMS[:,:K])
    beta2 = get_rowspace_projection(b2.T)
    return beta1, beta2  #np.dot(beta1,beta1.T), np.eye(d_x)-np.dot(beta1,beta1.T) 

def debias_proj(X_t, Y1_train, Y2_train, Iter, q, h):
    n = len(X_t)
    d = X_t.shape[1]
    X1 = X_t-np.mean(X_t,0).reshape(1,d)
    Cx = np.dot(X1.T,X1)/n
#     la, v = np.linalg.eig(Cx)
#     la = np.abs(la)
#     v = v.real
#     Cx12 = np.dot(np.dot(v,np.diag(la**(0.5))),v.T)
    InvC = np.linalg.pinv(Cx,rcond=1e-8)
    X_t = np.dot(X_t, InvC)

    X_ = X_t
    Y2_ = Y2_train
    beta1_g, beta2_g = SDR_multi_new(X_, Y2_, q, h)

    return beta1_g, beta2_g

In [21]:
def get_rowspace_projection(W: np.ndarray) -> np.ndarray:
    """
    :param W: the matrix over its nullspace to project
    :return: the projection matrix over the rowspace
    """
    if np.allclose(W, 0):
        w_basis = np.zeros_like(W.T)
    else:
        w_basis = scipy.linalg.orth(W.T) # orthogonal basis

    P_W = w_basis.dot(w_basis.T) # orthogonal projection on W's rowspace

    return P_W

In [22]:
def get_projection_to_intersection_of_nullspaces(rowspace_projection_matrices: List[np.ndarray], input_dim: int):
    """
    Given a list of rowspace projection matrices P_R(w_1), ..., P_R(w_n),
    this function calculates the projection to the intersection of all nullspasces of the matrices w_1, ..., w_n.
    uses the intersection-projection formula of Ben-Israel 2013 http://benisrael.net/BEN-ISRAEL-NOV-30-13.pdf:
    N(w1)∩ N(w2) ∩ ... ∩ N(wn) = N(P_R(w1) + P_R(w2) + ... + P_R(wn))
    :param rowspace_projection_matrices: List[np.array], a list of rowspace projections
    :param dim: input dim
    """
    I = np.eye(input_dim)
    Q = np.sum(rowspace_projection_matrices, axis = 0)
    P = I - get_rowspace_projection(Q)
    return P

In [23]:
clf_sem = LogisticRegression(warm_start = True, penalty = 'l2',
                         solver = "saga", multi_class = 'multinomial', fit_intercept = False,
                         verbose = 5, n_jobs = 90, random_state = 1, max_iter = 7)
clf_sem.fit(X_train, Y_sem_train)
print(clf_sem.score(X_test, Y_sem_test))

[Parallel(n_jobs=90)]: Using backend ThreadingBackend with 90 concurrent workers.


max_iter reached after 16 seconds
0.7119279819954989


[Parallel(n_jobs=90)]: Done   1 out of   1 | elapsed:   16.5s finished


In [24]:
Y = clf_sem.predict_proba(X_train)

In [25]:
clf_race = LogisticRegression(warm_start = True, penalty = 'l2',
                         solver = "saga", multi_class = 'multinomial', fit_intercept = False,
                         verbose = 5, n_jobs = 90, random_state = 1, max_iter = 7)
clf_race.fit(X_train, Y_race_train)
print(clf_race.score(X_test, Y_race_test))


[Parallel(n_jobs=90)]: Using backend ThreadingBackend with 90 concurrent workers.


max_iter reached after 16 seconds
0.8823455863965991


[Parallel(n_jobs=90)]: Done   1 out of   1 | elapsed:   16.4s finished


In [26]:
Z = clf_race.predict_proba(X_train)

In [27]:
Z

array([[3.86421321e-04, 9.99613579e-01],
       [8.88230709e-01, 1.11769291e-01],
       [9.95507796e-03, 9.90044922e-01],
       ...,
       [9.10343071e-01, 8.96569293e-02],
       [8.75272960e-01, 1.24727040e-01],
       [9.73272006e-01, 2.67279943e-02]])

In [28]:
h = 100

In [34]:
beta1, beta2 = debias_proj(X_train, 0, Z, 1, 1250, h)

In [35]:
pp = beta2

In [36]:
lr2 = LogisticRegression(warm_start = True, penalty = 'l2',
                         solver = "saga", multi_class = 'multinomial', fit_intercept = False,
                         verbose = 5, n_jobs = 90, random_state = 1, max_iter = 7)
lr2.fit(np.dot(X_train,pp), Y_sem_train)

accuracy = lr2.score(np.dot(X_test,pp), Y_sem_test)

Y_sem_pred = lr2.predict(np.dot(X_test,pp))

rms_diff_vec = get_TPR_gap(Y_sem_pred, Y_sem_test, Y_race_test)

Acc = accuracy*100
GAP = (1-rms_diff_vec)*100

[Parallel(n_jobs=90)]: Using backend ThreadingBackend with 90 concurrent workers.


max_iter reached after 16 seconds


[Parallel(n_jobs=90)]: Done   1 out of   1 | elapsed:   16.0s finished


In [37]:
Acc

69.00475118779696

In [38]:
rms_diff_vec

0.10569311467661427